In [2]:
import os
from dotenv import load_dotenv
import requests

# Load environment variables
load_dotenv()

api_key = os.getenv("STEAM_API_KEY")

# Example API request (get player summaries)
steam_id = os.getenv("STEAM_ID")
url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={api_key}&steamids={steam_id}"

response = requests.get(url)
data = response.json()

print(data)


{'response': {'players': [{'steamid': '76561199690294419', 'communityvisibilitystate': 3, 'profilestate': 1, 'personaname': 'brendan.rb', 'profileurl': 'https://steamcommunity.com/profiles/76561199690294419/', 'avatar': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270.jpg', 'avatarmedium': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270_medium.jpg', 'avatarfull': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270_full.jpg', 'avatarhash': '7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270', 'lastlogoff': 1762918447, 'personastate': 1, 'primaryclanid': '103582791429521408', 'timecreated': 1716259251, 'personastateflags': 0}]}}


In [5]:
import requests

# Get list of all Steam apps
url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
response = requests.get(url)
data = response.json()

# Example: print first 5 games
for app in data['applist']['apps'][:5]:
    print(app['appid'], app['name'])


5 Dedicated Server
7 Steam Client
8 winui2
10 Counter-Strike
20 Team Fortress Classic


In [8]:
len(data['applist']['apps'])

276723

In [4]:
appid = 570  # Example: Dota 2
url = f"https://store.steampowered.com/api/appdetails?appids={appid}"

response = requests.get(url)
data = response.json()

# Access the game info
game_data = data[str(appid)]['data']
print("Name:", game_data['name'])
print("Release date:", game_data['release_date']['date'])
print("Genres:", [g['description'] for g in game_data['genres']])
if 'price_overview' in game_data:
    print("Price:", game_data['price_overview']['final_formatted'])


Name: Dota 2
Release date: Jul 9, 2013
Genres: ['Action', 'Strategy', 'Free To Play']


In [9]:
import time
import pandas as pd
# Helper function to get app details from Steam Store API
def get_game_details(appid):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    response = requests.get(url)
    data = response.json()
    
    if data[str(appid)]['success']:
        game = data[str(appid)]['data']
        genres = [g['description'] for g in game.get('genres', [])]
        release_date = game.get('release_date', {}).get('date', None)
        developers = game.get('developers', [])
        publishers = game.get('publishers', [])
        price = game.get('price_overview', {}).get('final_formatted', None)
        metacritic = game.get('metacritic', {}).get('score', None)
        return {
            "appid": appid,
            "name": game.get('name'),
            "release_date": release_date,
            "genres": genres,
            "developers": developers,
            "publishers": publishers,
            "price": price,
            "metacritic_score": metacritic
        }
    return None

# Step 1: Get a list of all apps
app_list_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
resp = requests.get(app_list_url)
all_apps = resp.json()['applist']['apps']

# Step 2: For simplicity, pick the first 1000 apps and filter manually
# (You can replace this with top-sellers scraping if desired)
popular_apps = all_apps[:1000]  

games_data = []

for i, app in enumerate(popular_apps):
    try:
        details = get_game_details(app['appid'])
        if details:
            games_data.append(details)
            print(f"{i+1}: Fetched {details['name']}")
        time.sleep(0.5)  # rate limiting
        if len(games_data) >= 300:  # stop after 300 games
            break
    except Exception as e:
        print(f"Error fetching appid {app['appid']}: {e}")

# Step 3: Save to CSV
df = pd.DataFrame(games_data)
# df.to_csv("steam_top_300_games.csv", index=False)
# print("Saved top 300 Steam games to steam_top_300_games.csv")


4: Fetched Counter-Strike
5: Fetched Team Fortress Classic
6: Fetched Day of Defeat
7: Fetched Deathmatch Classic
8: Fetched Half-Life: Opposing Force
9: Fetched Ricochet
10: Fetched Half-Life
11: Fetched Counter-Strike: Condition Zero
13: Fetched Codename Gordon
14: Fetched Counter-Strike: Condition Zero
15: Fetched Half-Life: Blue Shift
16: Fetched Counter-Strike Beta
22: Fetched Half-Life 2: Demo
23: Fetched Half-Life 2
24: Fetched Counter-Strike: Source
26: Fetched Half-Life: Source
27: Fetched Day of Defeat: Source
30: Fetched Half-Life 2: Deathmatch
31: Fetched Half-Life 2: Lost Coast
32: Fetched Half-Life Deathmatch: Source
34: Fetched Half-Life 2: Episode One
35: Fetched Portal
36: Fetched Portal: First Slice
37: Fetched Half-Life 2: Episode Two
38: Fetched Team Fortress 2
44: Fetched Left 4 Dead
50: Fetched Left 4 Dead 2
55: Fetched Dota 2
65: Fetched Portal 2
67: Fetched Alien Swarm
76: Fetched Portal 2
77: Fetched Portal 2 Sixense MotionPack DLC
79: Fetched Counter-Strike 2


In [10]:
df.head()

,appid,name,release_date,genres,developers,publishers,price,metacritic_score
0,10,Counter-Strike,"Nov 1, 2000",[Action],[Valve],[Valve],$9.99,88.0
1,20,Team Fortress Classic,"Apr 1, 1999",[Action],[Valve],[Valve],$4.99,NaN
2,30,Day of Defeat,"May 1, 2003",[Action],[Valve],[Valve],$4.99,79.0
3,40,Deathmatch Classic,"Jun 1, 2001",[Action],[Valve],[Valve],$4.99,NaN
4,50,Half-Life: Opposing Force,"Nov 1, 1999",[Action],[Gearbox Software],[Valve],$4.99,NaN


In [12]:
df.to_parquet("steam_top_300_games.parquet", index=False)
print("Saved top 300 Steam games to steam_top_300_games.parquet")

Saved top 300 Steam games to steam_top_300_games.parquet
